Import numpy and pandas then read the Data

In [1]:
#--------------------------------------------------------------------------------- Movie Categories by Yasin Industries ---------------------------------------------------------------------------------

#Import pandas and numpy
import pandas as pd
import numpy as np

#Defing Id for all columns
ratingColumn = ['User_Id', 'Movie_Id', 'Rating']

#Import Datas
ratingData = pd.read_csv('Movie_rate.txt', sep='\t', names = ratingColumn, usecols=range(3))
ratingData.head(10)

,User_Id,Movie_Id,Rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3
5,22,377,1
6,244,51,2
7,166,346,1
8,298,474,4
9,115,265,2


In [2]:
#Show Ratings Size and mean
movieProperties = ratingData.groupby('Movie_Id').agg({'Rating':[np.size, np.mean]})
movieProperties.head(10)

Rating          
           size      mean
Movie_Id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326
6            26  3.576923
7           392  3.798469
8           219  3.995434
9           299  3.896321
10           89  3.831461

In [3]:
#Normakize Datas 
movieSizeRating = pd.DataFrame(movieProperties['Rating']['size'])
movieNormalized = movieSizeRating.apply(lambda x: ((x - np.min(x)) / (np.max(x) - np.min(x))))
movieNormalized.head(7)

,size
Movie_Id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798
6,0.042882
7,0.670669


In [4]:
#Read and export datas from the main dataset
dictionaryMovie = {}
with open(r'C:/Users/user/OneDrive/Documents/Yind_AI/Artificial Inteligence/Movie_item.txt') as f:
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieId = int(fields[0])
        name = fields[1]
        movieGenre = [int(x) for x in fields[5:25]]
        genre = map(int, movieGenre)
        dictionaryMovie[movieId] = (name, movieGenre,  movieNormalized.loc[movieId].get('size'), movieProperties.loc[movieId].Rating.get('mean'))


In [7]:
#Calculate the distance of 2 movies
from scipy import spatial as sp

#Calculate Genre and Rating amount distance
def calculateDistance(firstMovie, secondMovie):
    genreFirstMovie = firstMovie[1] 
    genreSecondMovie = secondMovie[1]
    genreDistance = sp.distance.cosine(genreFirstMovie, genreSecondMovie)
    firstRatingSize = firstMovie[2]
    secondRatingSize = secondMovie[2]
    ratingSizeDistance = abs(firstRatingSize - secondRatingSize)
    return (f'Distance : {genreDistance}')

print(calculateDistance(dictionaryMovie[2], dictionaryMovie[5]))

Distance : 0.6666666666666667


In [6]:
#Make a function for showing the nearest movies
import operator as op
#k = movie numbers
def nearestMovie(movieKey, k):
    distance = []
    near = []
    for movie in dictionaryMovie:
        if movie != movieKey:
            Dist = calculateDistance(dictionaryMovie[movie], dictionaryMovie[movieKey])
            distance.append((movie, Dist))
    distance.sort(key=(op.itemgetter(1)))
    for x in range(k):
        near.append(distance[x][0])
    return near



In [7]:
k = 5
aveRating = 0
sameMovies = nearestMovie(1, k) #first item is the movie Id (1 => Toy Story)
for sameMovies in sameMovies:
    aveRating += dictionaryMovie[sameMovies][3]
    print(f'{dictionaryMovie[sameMovies][0]}')

aveRating /= float(k)
print("-------------------------------")
print(aveRating)


Aladdin and the King of Thieves (1996)
Aladdin (1992)
Goofy Movie, A (1995)
Santa Clause, The (1994)
Home Alone (1990)
-------------------------------
3.148818290153474


C:\ProgramData\Anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [8]:
print(dictionaryMovie)

{1: ('Toy Story (1995)', [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.7735849056603774, 3.8783185840707963), 2: ('GoldenEye (1995)', [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.22298456260720412, 3.2061068702290076), 3: ('Four Rooms (1995)', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.15265866209262435, 3.033333333333333), 4: ('Get Shorty (1995)', [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.3567753001715266, 3.550239234449761), 5: ('Copycat (1995)', [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.1457975986277873, 3.302325581395349), 6: ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.04288164665523156, 3.576923076923077), 7: ('Twelve Monkeys (1995)', [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 0.6706689536878216, 3.798469387755102), 8: ('Babe (1995)', [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.37392795883361923, 3.995433789954337